# Definitions (functions and subroutines)

With all of the modules that we have been using this semester you have been utilizing the functionality of python definitions. A definition is the ability to write a generic function that you can use more than once in a program without having write it multiple times in the same program (or write in multiple different programs.

Today we'll begin with some simple examples of how to write your own functions, then we'll move on to a great example that will help us find index values for various lat/lon points and then we'll talk about how to create and load a scripts that contains lots of common functions for you!

## Writing a definition

A common thing to write up in a definition would be a unit coversion or simple mathematical calculation. For example, we can write a definition to calculate the area of a square. In that case the required mathematical function is

area = length * width

So let's write a definition to calculate that area.

In [1]:
def area_rect(l,w):
    '''This function calculates the area of a rectangular object using
           area = length * width
           l is the length
           w is the width
           
           returns area
    '''
    area = l*w
    return area

Let's break that definition down a little

**def** is the keyword for declaring a definition (e.g., function or subroutine)

**area_rect** is the actual name of the function that you would use in your program to calculate the area

**(l,w)** are the inputs for the function. You can feed the function particular values like 8 and 4 or a variable that represents a length and width of a rectangle.

The **:** works similarly to the colon in the for loop and if statement. Everything after the colon should be indented so that python interprets it to be a part of the definition.

The three quote marks defines the doc string that explains what the function is and how it works. If you were to type area_rect? into a cell you would get back that doc string.

**area = l * w ** is the actual calculation being performed in the definition

**return area** is what gets returned to the main program.

In [ ]:
area_rect?

So once you have run the cell that contains the function it is defined for our use in this IPython Notebook. But it is just generic code, sitting there, not really doing anything. We need to call it and feed it some values for it to do work and offer some output.

In [2]:
print(area_rect(8,4))

32


In [3]:
length = 9.
width = 5.
a = area_rect(length,width)
print(a)

45.0


The variables that you feed into the function do not have to be the same as what they are called in the function. Actually, you are better served by them NOT being the same. Mainly so you don't confuse yourself as to what is the value of a particular variable at any given point in your program.

Interestingly enough you could feed an array into the length and/or width and it would give you out an array of areas.

In [4]:
import numpy as np
length = np.arange(1,16)
width = np.arange(15,0,-1)
b = area_rect(length,width)
print(b.shape)
print(b)

(15,)
[15 28 39 48 55 60 63 64 63 60 55 48 39 28 15]


## More complex function

We have needed to find the index value corresponding to a particular latitude or longitude a couple of times this semester. This is the perfect example of something you may want to write a function to help you determine, mainly so that you can write the function once and then just use it anytime you need it.

We'll begin by putting a previous example of finding a longitude index value for a given longitude value in degrees and then work on constructing a function from there.

In [5]:
lats = np.loadtxt('lat.dat')
lons = np.loadtxt('lon.dat')

# Do the same for longitude
dlon = np.abs(lons - 151.2)
ilon = np.where(dlon == np.min(dlon))[0][0]
print(ilon,lons[ilon])

60 150.0


In [6]:
def lat_lon_index(L,value):
    '''This function will determine which index value of the 
       given array corresponds to the closest grid point to 
       the desired latitude or longitude point.
    
       L is the array values of latitude or longitude from the gridded data
       L must be in degrees East from 0 - 360 for NCEP/NCAR reanalysis data
       
       value is the latitude or longitude value of the point 
       that you want to find the closest gridpoint
       
       returns the index value of the array element of the 
       closest value to the given input lat/lon
    '''
    dL = np.abs(L - value)
    return np.where(dL == np.min(dL))[0][0]

You can make this function a lot more sophisticated with conducting some checks or incorporating a keyword for whether you are searching for a latitude or longitude, but this is at least a start and a good example of how you can use a function to help you accomplish your tasks.

In [7]:
ilon = lat_lon_index(lons,151.2)
print(ilon,lons[ilon])

60 150.0


In [8]:
ilat = lat_lon_index(lats,-34)
print(ilat,lats[ilat])

50 -35.0


Let's see how we could really use this function to a great advantage with an example from the previous class. We were trying to define a box that would subset the NCEP/NCAR reanalysis data to be just over the CONUS, but we needed to determine what those index values would be for the lower-left (LL) and upper-right (UR) coordinate values. I gave the following:

**Actual Lat/Lon Values**

LLlon = -130 <br>
LLlat = 20 <br>
URlon = -60 <br>
URlat = 60 <br>

**Corresponding index values**

iLLlon = 92 <br>
iLLlat = 28 <br>
iURlon = 120 <br>
iURlat = 12 <br>

In the next cell use the previously defined function to find the index values given the actual lat/lon values

In [9]:
# Here are the actual lat/lon values
LLlon = -130
LLlat = 20
URlon = -60
URlat = 60

# Write some function calls here
iLLlon = lat_lon_index(lons,LLlon+360)
iLLlat = lat_lon_index(lats,LLlat)
iURlon = lat_lon_index(lons,URlon+360)
iURlat = lat_lon_index(lats,URlat)
print(iLLlon,iLLlat)
print(iURlon,iURlat)

92 28
120 12


## Two Dimensional lat and lon arrays

It is a more difficult process for finding the closest grid point when the lat and lon arrays are two dimensional. Below is a function that I have written to calculate the distance between the desired point and every single grid point within the array to ultimately find the array element that has the smallest distance from the desired point.

First, let's make some 2D lat and lon arrays

In [10]:
clons, clats = np.meshgrid(lons,lats)

In [11]:
# Find lat_lon_index for a 2D lat/lon array
def lat_lon_2D_index(y,x,lat1,lon1):
    '''This function calculates the distance from a desired 
    lat/lon point to each element of a 2D array of lat/lon values,
    typically from model output, and determines the index value 
    corresponding to the nearest lat/lon grid point.
    
    x = longitude array
    y = latitude array
    lon1 = longitude point (signle value)
    lat1 = latitude point (single value)
    
    Returns the two index values for nearest lat, lon 
    point on grid for grids (y,x)
    
    Equations for variable distiance between longitudes from
    http://andrew.hedges.name/experiments/haversine/'''
    
    R = 6373.*1000.  # Earth's Radius in meters
    rad = np.pi/180.
    x1 = np.ones(x.shape)*lon1
    y1 = np.ones(y.shape)*lat1
    dlon = np.abs(x-x1)
    dlat = np.abs(y-y1)
    a = (np.sin(rad*dlat/2.))**2 + 
         np.cos(rad*y1) * np.cos(rad*y) * (np.sin(rad*dlon/2.))**2 
    c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a) ) 
    d = R * c
    print(d.shape)
    return np.unravel_index(d.argmin(), d.shape)

In [12]:
# Here are the actual lat/lon values
LLlon = -130
LLlat = 20
URlon = -60
URlat = 60

# Write some function calls here
iLLlat,iLLlon = lat_lon_2D_index(clats,clons,LLlat,LLlon)
iURlat,iURlon = lat_lon_2D_index(clats,clons,URlat,URlon)

print(iLLlon,iLLlat)
print(iURlon,iURlat)

(73, 144)
(73, 144)
92 28
120 12


## Temp Converstion Function

Now that you know how functions work, go ahead and write your own function to convert temperatures in Fahrenheit to Celsius

In [13]:
def tmpf2tmpc(temp):
    '''
    This function converts a temperature in Fahrenheit to Celsius
    
    tc = (5/9)*(tf - 32)
    '''
    tmpc = (5./9.)*(temp - 32)
    return tmpc

In [14]:
print(tmpf2tmpc(50))

10.0


## Collecting Functions

You can collect some of your most commonly used functions in a python script and import it just like you do modules like numpy, matplotlib, and netCDF4. Those modules are simply sophisticated groups of classes and functions that are installed on your system.

If you had a python script that contained your own functions called "myfuncs.py" you can import it into any script or ipython notebook with the following import call

**import myfuncs**

provided that the myfuncs.py file is located in the directory you are working in or running the script from. You could also place a copy where all of your modules reside, then you wouldn't have to worry about having it in your working directory, but that is a bit beyond us at this point.